# 30 - ACS 5-year Tracts

Pull a minimal set of ACS variables for TX/LA/MS/AL/FL; write CSV.


In [1]:
import os, requests, pandas as pd
from pathlib import Path

ROOT = Path('/Users/liamguest/LProjects/AURA/AURA')
INT = ROOT / 'data' / 'interim'; INT.mkdir(parents=True, exist_ok=True)

STATE_FIPS = ['48','22','28','01','12']
CENSUS_API_KEY = os.getenv('CENSUS_API_KEY')
print('Has key:', bool(CENSUS_API_KEY))

BASE = 'https://api.census.gov/data/2022/acs/acs5'
vars_ = ['NAME','B01003_001E','B19013_001E']
get = ','.join(['GEO_ID'] + vars_)

frames = []
for st in STATE_FIPS:
    params = {'get': get, 'for': 'tract:*', 'in': f'state:{st}'}
    if CENSUS_API_KEY:
        params['key'] = CENSUS_API_KEY
    r = requests.get(BASE, params=params, timeout=60)
    r.raise_for_status()
    df = pd.DataFrame(r.json()[1:], columns=r.json()[0])
    df['state_fips'] = st
    frames.append(df)

acs = pd.concat(frames, ignore_index=True)
out = INT / 'acs_2022_tx_la_ms_al_fl.csv'
acs.to_csv(out, index=False)
print('Wrote', out, 'rows:', len(acs))


Has key: False
Wrote /Users/liamguest/LProjects/AURA/AURA/data/interim/acs_2022_tx_la_ms_al_fl.csv rows: 15759


## Clean and standardize ACS

- Build 11-digit `tract_geoid` from `GEO_ID`
- Coerce numerics
- Rename to readable names
- Save cleaned CSV


In [2]:
import pandas as pd
from pathlib import Path

INT = Path('/Users/liamguest/LProjects/AURA/AURA/data/interim')
PROC = Path('/Users/liamguest/LProjects/AURA/AURA/data/processed'); PROC.mkdir(parents=True, exist_ok=True)

acs_path = INT / 'acs_2022_tx_la_ms_al_fl.csv'
acs = pd.read_csv(acs_path)
print('Loaded ACS:', acs.shape)

# GEO_ID like: '1400000US48157671202' → tract geoid is last 11 chars
acs['tract_geoid'] = acs['GEO_ID'].astype(str).str[-11:]

# Coerce numeric columns
for col in ['B01003_001E','B19013_001E']:
    if col in acs.columns:
        acs[col] = pd.to_numeric(acs[col], errors='coerce')

# Rename to readable
rename_map = {
    'B01003_001E': 'total_population',
    'B19013_001E': 'median_household_income'
}
acs = acs.rename(columns=rename_map)

# Keep essential columns
keep = ['tract_geoid','NAME','state_fips','total_population','median_household_income']
acs_clean = acs[[c for c in keep if c in acs.columns]].copy()

out_clean = PROC / 'acs_2022_tx_la_ms_al_fl_clean.csv'
acs_clean.to_csv(out_clean, index=False)
print('Wrote', out_clean, 'rows:', len(acs_clean))


Loaded ACS: (15759, 8)
Wrote /Users/liamguest/LProjects/AURA/AURA/data/processed/acs_2022_tx_la_ms_al_fl_clean.csv rows: 15759
